In [1]:
#Correr solo si se va a plotear spectrogramas

import librosa

def plot_spectrogram(specgram, title=None, ylabel="freq_bin", ax=None):
    if ax is None:
        _, ax = plt.subplots(1, 1)
    if title is not None:
        ax.set_title(title)
    ax.set_ylabel(ylabel)
    ax.imshow(librosa.power_to_db(specgram), origin="lower", aspect="auto", interpolation="nearest")

In [2]:
import os
import pandas as pd
from torch import nn
from torch.utils.data import DataLoader
import torch
from tqdm import tqdm
import matplotlib.pyplot as plt
from torchsummary import summary



In [3]:
#librerias locales
#from ..libs import dataLoaders as mi_dataLoader
#from ..libs import clasificador_padre as mi_clasificador
from importlib.machinery import SourceFileLoader
mi_dataLoader = SourceFileLoader('dataLoadersV2', '../libs/dataLoadersV2.py').load_module()
mi_clasificador = SourceFileLoader('clasificador_padre', '../libs/clasificador_padre.py').load_module()

# Cargando un DataSet

In [4]:
dataSetName = 'onlywoman'
root = './'
while not os.path.isdir(f'{root}data'):
    root += '../'
Path_train = root+'data/'+dataSetName+'_train.csv'
Path_val = root+'data/'+dataSetName+'_val.csv'
Path_test = root+'data/'+dataSetName+'_test.csv'

In [5]:
#cargo el data set
testDf = pd.read_csv(Path_train)
testDataSet = mi_dataLoader.clasification_dataset(testDf, file_col="fileMfcc", target_col="artist", seed = 98)
#una muestra del data set
sample, target = testDataSet[0:9]
print(sample[0].shape, target)

100%|██████████| 8799/8799 [00:02<00:00, 3401.16it/s]

torch.Size([13, 997]) [tensor([1., 0., 0.]), tensor([1., 0., 0.]), tensor([1., 0., 0.]), tensor([1., 0., 0.]), tensor([1., 0., 0.]), tensor([1., 0., 0.]), tensor([1., 0., 0.]), tensor([1., 0., 0.]), tensor([1., 0., 0.])]


# Generando el DataLoader

In [6]:
test_dataloader = DataLoader(testDataSet, batch_size=500, shuffle=True)
#muestra_data, muestra_target = next(iter(test_dataloader))
#muestra_target

In [7]:
###Muestras de especrogramas

#train_features, train_labels = next(iter(test_dataloader))
#print(f"Feature batch shape: {train_features.size()}")
#print(f"Labels batch shape: {train_labels.size()}")
#plot_spectrogram(train_features[0], train_labels[0])
#img = train_features[0].squeeze()
#label = train_labels[0]
#plt.imshow(img, cmap="gray")
#plt.show()
#print(f"Label: {label}")

# instanciando un modelo

In [8]:
os.listdir('./save/state')
pState = "./save/state/"
pHist = "./save/history/"
overWrite = False
show_metrics = True
sLoad = 4 #save state a cargar
lr = 0.0001
constructor = mi_clasificador.onlyWoman_MFCC_05_N2

In [9]:
def test_accur (prediccion, target_batch):
    return(torch.argmax(target_batch, axis=1) == torch.argmax(prediccion, axis=1)).sum().item() / len(target_batch)


if((len(os.listdir(pState)) == 0) or overWrite):
    history = {}
    history['loss'] = torch.empty(0)
    history['acur'] = torch.empty(0)

    clasificador = constructor(list(testDataSet.dictionary.keys()))
else:
    history =  torch.load(pHist+os.listdir(pHist)[sLoad])

    save_state = torch.load(pState+os.listdir(pState)[sLoad])
    clasificador = constructor(list(testDataSet.dictionary.keys()))
    clasificador.load_state_dict(save_state)

test_optimizer = torch.optim.Adam(clasificador.parameters(), lr=lr)
test_criterion = torch.nn.CrossEntropyLoss()

#envio el modelo a la grafica si esta disponible
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"Hay {torch.cuda.device_count()} GPU(s) disponibles.")
    print(f"GPU activa: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")
    #print("CUDA no está disponible, se utilizará la CPU.")
clasificador.to(device)
print(clasificador.modules)



Hay 1 GPU(s) disponibles.
GPU activa: NVIDIA GeForce RTX 4050 Laptop GPU
<bound method Module.modules of onlyWoman_MFCC_05_N2(
  (inPut): Sequential(
    (0): Conv2d(1, 4, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
  )
  (chanelUp): Sequential(
    (0): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): ReLU()
    (2): Conv2d(4, 16, kernel_size=(1, 7), stride=(1, 1))
    (3): ReLU()
    (4): Conv2d(16, 32, kernel_size=(1, 7), stride=(1, 1))
    (5): ReLU()
    (6): Conv2d(32, 64, kernel_size=(1, 7), stride=(1, 2))
    (7): ReLU()
    (8): MaxPool2d(kernel_size=(1, 2), stride=(1, 2), padding=0, dilation=1, ceil_mode=False)
    (9): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (justTime): Sequential(
    (0): Conv2d(64, 64, kernel_size=(3, 7), stride=(1, 2))
    (1): ReLU()
    (2): Conv2d(64, 64, kernel_size=(3, 7), stride=(1, 2))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=(1, 2), s

# Entrenando el modelo

In [10]:
show_metrics = False

In [13]:
%matplotlib qt5

if show_metrics:
    plt.ion()
    fig, axes = plt.subplots(nrows=2, ncols=1, figsize = (6,6))
    l1, = axes[0].plot([], [])
    l2, = axes[1].plot([], [])
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Acur')

    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('Loss')

for i in range(10):
    loss_log, acur_log = clasificador.train_loop(1, test_dataloader, test_optimizer , test_criterion, test_accur, device)
    history['loss'] = torch.cat((history['loss'], loss_log), dim=0)
    history['acur'] = torch.cat((history['acur'], acur_log), dim=0)
    
    if show_metrics:
        # Grafica
        l1.set_data(range(len(history['acur'])), history['acur'])
        axes[0].relim()  # Recalculate data limits
        axes[0].autoscale_view()  # Rescale axes

        l2.set_data(range(len(history['loss'])), history['loss'])
        axes[1].relim()  # Recalculate data limits
        axes[1].autoscale_view()  # Rescale axes
        
        fig.canvas.draw()  # Redraw the figure
        fig.canvas.flush_events()  # Ensure events are processed
        plt.pause(0.5)

save_state = clasificador.state_dict()
n = len(os.listdir(pState))
torch.save(save_state, f"{pState}/ss_{n}.pt")
torch.save(history, f"{pHist}/hist_{n}.pt")

if show_metrics:
    plt.ioff()
    plt.show()

Epoch 0 of 1 / Loss: 0.5994521975517273 / Accuracy: 0.9542876482009888
Epoch 0 of 1 / Loss: 0.5918563604354858 / Accuracy: 0.9611018300056458
Epoch 0 of 1 / Loss: 0.586625337600708 / Accuracy: 0.9658796191215515
Epoch 0 of 1 / Loss: 0.5821231603622437 / Accuracy: 0.9704024791717529
Epoch 0 of 1 / Loss: 0.5788730978965759 / Accuracy: 0.9730327129364014
Epoch 0 of 1 / Loss: 0.5762735605239868 / Accuracy: 0.9757740497589111
Epoch 0 of 1 / Loss: 0.57389235496521 / Accuracy: 0.9778105020523071
Epoch 0 of 1 / Loss: 0.5725435614585876 / Accuracy: 0.9790728092193604
Epoch 0 of 1 / Loss: 0.5715665817260742 / Accuracy: 0.9799617528915405
Epoch 0 of 1 / Loss: 0.5714296698570251 / Accuracy: 0.9801437854766846


# Validando

In [15]:
def acuracy_fn (prediccion, target_batch):
    return(torch.argmax(target_batch, axis=1) == torch.argmax(prediccion, axis=1)).sum().item() / len(target_batch)


#Carga de las muestras y el DataLoader
valDf = pd.read_csv(Path_val)
valDataSet = mi_dataLoader.clasification_dataset(valDf, file_col="fileMfcc", target_col="artist", seed = 98)
val_dataloader = DataLoader(valDataSet, batch_size=100, shuffle=False)

#Separando las muestras y los targets
muestras = []
targets = []
for muestra_data, muestra_target in tqdm(val_dataloader):
    muestras.append(muestra_data)
    targets.append(torch.tensor(muestra_target))

loss_fn = torch.nn.CrossEntropyLoss()
resultado_evalucion = clasificador.evaluate(val_dataloader, loss_fn, acuracy_fn, device)
print(resultado_evalucion)


  0%|          | 0/27 [00:00<?, ?it/s]C:\Users\leona\AppData\Local\Temp\ipykernel_26192\3220346858.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  targets.append(torch.tensor(muestra_target))
100%|██████████| 27/27 [00:03<00:00,  7.20it/s]

(0.6613621027381332, 0.8881481481481482)


A continuacion se levantan los distintos save state generados para generar una comparacion entre las distintas performance en el validation set

In [14]:
val_histo = []
max_acur = 0
mejor = None
for ss_name in os.listdir(f"{pState}/"):
    ss_load = torch.load(f"{pState}{ss_name}")
    clasificador.load_state_dict(ss_load)
    resultado_evalucion = clasificador.evaluate(val_dataloader, loss_fn, acuracy_fn, device)
    val_histo.append(resultado_evalucion)
    if resultado_evalucion[1] > max_acur:
        max_acur = resultado_evalucion[1]
        mejor = ss_load
val_histo

100%|██████████| 27/27 [00:03<00:00,  6.84it/s]


[(0.6752566871819673, 0.8790740740740742),
 (0.6613621027381332, 0.8881481481481482)]

# Test set

In [53]:
#Carga de las muestras y el DataLoader
testDf = pd.read_csv(Path_test)
testDataSet = mi_dataLoader.clasification_dataset(testDf, file_col="fileMfcc", target_col="artist", seed = 98)
test_dataloader = DataLoader(testDataSet, batch_size=100, shuffle=False)

clasificador.load_state_dict(mejor)
clasificador.evaluate(test_dataloader, loss_fn, acuracy_fn, device)

100%|██████████| 3/3 [00:00<00:00,  6.33it/s]


(0.6555488705635071, 0.887872340425532)

## Codigo usado mientras se generaba la arquitectura (osea, codigo desechable)

In [ ]:
mi_clasificador = SourceFileLoader('clasificador_padre', '../libs/clasificador_padre.py').load_module()

In [24]:
muestra_data, muestra_target = next(iter(test_dataloader))
#muestra = torch.stack(muestra, 0)
muestra_data.shape

torch.Size([100, 13, 997])

In [59]:
mi_clasificador = SourceFileLoader('clasificador_padre', '../libs/clasificador_padre.py').load_module()
clasificador = mi_clasificador.onlyWoman_MFCC_01(list(testDataSet.dictionary.keys()))

In [60]:
res =clasificador(muestra_data)
res.shape

In: torch.Size([100, 13, 997])
squeeze: torch.Size([100, 1, 13, 997])
Shape post 1D:  torch.Size([100, 4, 15, 999])
post flatten: torch.Size([100, 136136])
Alive


torch.Size([100, 3])

In [45]:
out = nn.Sequential(   # bloque que acondiciona la salida
    nn.Linear(*self.architecture[-1]), #Lineal que sale a las clases objetivo
    nn.Softmax( dim = 1))

NameError: name 'self' is not defined

In [61]:
test_criterion(res,muestra_target)

tensor(1.1800, grad_fn=<DivBackward1>)

In [62]:
test_accur (res, muestra_target)

0.3

In [64]:
#importlib.reload(mi_clasificador)
clasificador = mi_clasificador.Clasificador_01(testDataSet.dictionary.keys())
res =clasificador(muestra_data)
test_accur (res, muestra_target)


TypeError: _Clasificador_padre.__init__() missing 1 required positional argument: 'classes'

In [318]:
(torch.argmax(muestra_target, axis = 1) == torch.argmax(res, axis=1)).sum().item() / len(muestra_target)

0.0

In [319]:
torch.argmax(muestra_target, axis = 1) == torch.argmax(res, axis=1)

tensor([False])

In [42]:
muestra_target.shape, res.shape

(torch.Size([3, 3]), torch.Size([3, 3]))

In [43]:
torch.argmax(res, axis = 1)

tensor([0, 2, 2])

In [44]:
res

tensor([[9.9844e-01, 2.4358e-21, 1.5574e-03],
        [1.7411e-05, 4.4522e-07, 9.9998e-01],
        [0.0000e+00, 0.0000e+00, 1.0000e+00]], grad_fn=<SoftmaxBackward0>)

In [43]:
torch.argmax(muestra_target, axis=1)

tensor([0, 0, 0, 0, 0])

In [44]:
muestra_target

tensor([[1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.]])

In [21]:
muestra_target == res

tensor([[False,  True, False],
        [False,  True, False],
        [False, False, False],
        [False,  True, False],
        [False, False, False]])

In [45]:
res

tensor([[0.0000e+00, 4.1355e-15, 1.0000e+00],
        [0.0000e+00, 0.0000e+00, 1.0000e+00],
        [0.0000e+00, 1.0000e+00, 4.7325e-08],
        [0.0000e+00, 1.0000e+00, 4.9665e-37],
        [1.1335e-33, 3.0150e-14, 1.0000e+00]], grad_fn=<SoftmaxBackward0>)

In [ ]:
flat = nn.Flatten(start_dim = 0)
lin = nn.Linear(17920,3)
soft = nn.Softmax(dim = 0)

print("Red: ",res.shape)
aux = flat(res)
print("Flat: ",aux.shape)
aux = lin(aux)
print("Line: ",aux.shape)
aux = soft(aux)
print("Soft:",aux.shape)

Red:  torch.Size([128, 5, 28])
Flat:  torch.Size([17920])
Line:  torch.Size([3])
Soft: torch.Size([3])
